In [ ]:
import gromacs as gmx
from gromacs import formats as gmf
import numpy as np
import mdtraj as md
import nglview as nv

In [ ]:
dir='/home/meta/ljocha/ASMSA/My_AE/trpcage/ae'
ftrain=dir + '/trpcage_ds.npy'
pdb = dir + '/trpcage_npt400.pdb'
xtc = dir + '/trpcage_ds_fit.xtc'
dec = '/home/meta/ljocha/ASMSA/My_AE/trpcage/dec_out_LS_matrix_pos_32x32.npy'

train = np.load(ftrain)

In [ ]:
train_view = np.reshape(train,(-1,272,3))

In [ ]:
train.shape

In [ ]:
tr = md.load(pdb)

In [ ]:
tr.xyz = train_view

In [ ]:
v=nv.show_mdtraj(tr)
v

In [ ]:
dec_out_all = np.load(dec)
dec_out_all.shape

In [ ]:
def Unstandardize(x):
    x = np.array(x)
    return ((x*7)-1)

In [ ]:
dec_out_all = Unstandardize(dec_out_all)

In [ ]:
dec_out = dec_out_all[6,18]

In [ ]:
diff = train - dec_out
msd = np.sum(diff * diff,axis=1)

In [ ]:
imin = np.argmin(msd)
imin

In [ ]:
tr2 = md.load(pdb)
tr2.xyz = np.reshape(dec_out_all,(-1,272,3))
#tr2.xyz = train_view[9873]
v2 = nv.show_mdtraj(tr2)
#v2.clear()
v2.add_representation("licorice")
v2

In [ ]:
v.frame=int(imin)

In [ ]:
tr3 = md.load(xtc,top=pdb)
tr3.xyz = np.reshape(dec_out_all,(-1,272,3))
tr3.time = tr3.time[0:tr3.xyz.shape[0]]
tr3.unitcell_vectors = tr3.unitcell_vectors[0:tr3.xyz.shape[0]]

tr3.save_xtc('simulated.xtc')

### Identify backbone atoms

In [ ]:
!cp {pdb} reference.pdb

In [ ]:
gmx.select(s='reference.pdb',on='bb.ndx',select='Backbone')

### Compute backbone dihedrals of the simulated trajectory

In [ ]:
with open('bb.ndx') as bb:
    bb.readline()
    l = " ".join(bb.readlines())
    ndx = np.array(l.split(),np.int32)

In [ ]:
with open('angle.ndx','w') as a:
    a.write('[ dihedrals ]\n')
    for i in range(0,len(ndx)-3):
        a.write('  '.join(map(str,ndx[i:i+4])))
        a.write('\n')

In [ ]:
gmx.gangle(f='simulated.xtc',n='angle.ndx',g1='dihedral',group1='dihedrals',oall='dihedrals.xvg')

In [ ]:
dihs = gmf.XVG('dihedrals.xvg').array[1:]

### Generate restrained topology and run minimization

In [ ]:
gmx.pdb2gmx(f='reference.pdb',o='reference.gro',p='reference.top',water='tip3p',ff='amber99')

In [ ]:
with open('min.mdp','w') as m:
    m.write('''
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

nstxout                 = 1         
nstvout                 = 0         
nstfout                 = 0         
nstlog                  = 5
nstxout-compressed      = 1

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = Cut-off   ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

disre           = Simple
define                  = -DPOSRES 
''')

In [ ]:
%%time 
n = 123
for n in range(123,124):
    with open('posre.itp','w') as p:
        p.write('[ dihedral_restraints ]\n')
        for i in range(0,len(ndx)-3):
            p.write('  '.join(map(str,ndx[i:i+4])))
            p.write(f' 1 {dihs[i,n]} 0 5000\n')
    
    gtr = tr3[n]
    gtr.save_gro('min.gro')
    
    gmx.grompp(f='min.mdp',c='min.gro',p='reference.top',o='min.tpr',maxwarn=100000)
    
    gmx.mdrun(deffnm='min')

In [ ]:
v=nv.show_file('min.gro')
v.add_representation('licorice')
v

In [ ]:
v2=nv.show_mdtraj(tr3[n])
v2.add_representation('licorice')
v2

In [ ]:
gmx.gangle(f='min.gro',n='angle.ndx',g1='dihedral',group1='dihedrals',oall='min.xvg')

In [ ]:
!cat min.xvg

In [ ]:
dihs[:,n]

In [ ]:
gmx.energy(f='min.edr',input=['Potential','Dih.-Rest.'],o='emin.xvg')

In [ ]:
import matplotlib.pyplot as plt
energ = np.loadtxt('emin.xvg',comments=['#','@'])
plt.plot(energ[:,1],label='Dih.-Rest.')
plt.plot(energ[:,2],label='Potential')
plt.plot(energ[:,2]-energ[:,1],label='Net')
plt.yscale('log')
plt.legend()
plt.show()


In [ ]:
mtr = md.load('min.trr',top=pdb)
mtr.xyz = mtr.xyz[-1:0:-1]
v = nv.show_mdtraj(mtr)
v.add_representation('licorice')
v

In [ ]:
mtr.xyz.shape